# Deep Research Technique Evaluation on Math Benchmarks

This notebook comprehensively evaluates the **Test-Time Diffusion Deep Researcher (TTD-DR)** algorithm against challenging mathematical benchmarks and compares its performance with state-of-the-art AI models.

## Benchmarks Evaluated:
1. **FrontierMath** - Advanced mathematics research problems
2. **HARP** - Hard Arithmetic Reasoning Problems
3. **IMO-bench** - International Mathematical Olympiad problems
4. **AIME** - American Invitational Mathematics Examination
5. **MATH-500** - High-school competition mathematics

## SOTA Models Compared:
- ChatGPT (GPT-4/GPT-4.5)
- Gemini 2.5 Pro
- Claude 4.1 Opus
- Grok 4
- DeepSeek V3

## Evaluation Methodology:
- Run Deep Research approach on each benchmark
- Compare accuracy, reasoning depth, and computational efficiency
- Visualize performance differences
- Analyze strengths and weaknesses


## Setup and Imports


In [ ]:
# Install required packages
%pip install -q openai datasets huggingface-hub pandas numpy matplotlib seaborn plotly tqdm scikit-learn


In [ ]:
import sys
import os
import json
import time
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from openai import OpenAI
from datasets import load_dataset
from tqdm.notebook import tqdm

# Set plotting style
sns.set_theme(style="whitegrid", palette="husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✓ All imports successful!")
print(f"Current working directory: {os.getcwd()}")


## Configuration


In [ ]:
# Configure API endpoints
# For Deep Research (using OptiLLM server)
OPTILLM_BASE_URL = "http://localhost:8001/v1"
OPTILLM_API_KEY = "optillm"

# Initialize OpenAI client for OptiLLM
client = OpenAI(api_key=OPTILLM_API_KEY, base_url=OPTILLM_BASE_URL)

# Model configuration
BASE_MODEL = "gpt-4o-mini"  # Change to your preferred model

# Evaluation configuration
NUM_PROBLEMS_PER_BENCHMARK = 30  # Number of problems to test per benchmark
TIMEOUT_SECONDS = 600  # 10 minutes per problem
MAX_DEEP_RESEARCH_ITERATIONS = 5
MAX_SOURCES = 30

# Results directory
RESULTS_DIR = "deep_research_benchmark_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

print("✓ Configuration complete!")
print(f"Results will be saved to: {RESULTS_DIR}")


## Quick Start: Run the Complete Evaluation

The complete evaluation script is available as a standalone Python file. You can run it in two ways:

### Option 1: Run as a Python Script
```bash
cd /Users/wikiwoo/Desktop/optillm
python notebooks/deep_research_math_evaluation_complete.py
```

### Option 2: Execute from this Notebook
Run the cell below to execute the complete evaluation:


In [ ]:
# Execute the complete evaluation script
%run notebooks/deep_research_math_evaluation_complete.py


## Custom Evaluation: Step-by-Step

If you want to customize the evaluation or run specific benchmarks, use the cells below.


In [ ]:
# Import the complete evaluation module
sys.path.append('/Users/wikiwoo/Desktop/optillm/notebooks')
from deep_research_math_evaluation_complete import (
    load_aime_dataset,
    load_math500_dataset,
    load_imo_dataset,
    create_synthetic_frontier_math,
    create_synthetic_harp,
    evaluate_approach_on_benchmark,
    get_sota_performance_data
)

# Load a specific benchmark
print("Loading AIME 2024 dataset...")
aime_problems = load_aime_dataset(2024, limit=5)

print(f"\nLoaded {len(aime_problems)} AIME problems")
print("\nSample problem:")
print(f"ID: {aime_problems[0]['id']}")
print(f"Problem: {aime_problems[0]['problem'][:200]}...")
print(f"Answer: {aime_problems[0]['answer']}")


In [ ]:
# Evaluate a single approach on a single benchmark
# This will take a few minutes depending on the number of problems

print("Running evaluation of baseline approach on AIME...")
baseline_results = evaluate_approach_on_benchmark(
    aime_problems[:3],  # Evaluate only 3 problems for quick demo
    approach="none",
    benchmark_name="AIME-2024-Demo"
)

print("\n" + "="*80)
print("Baseline Results Summary:")
print("="*80)
print(f"Accuracy: {baseline_results['accuracy']:.1f}%")
print(f"Correct: {baseline_results['correct']}/{baseline_results['total_problems']}")
print(f"Avg Tokens: {baseline_results['avg_tokens']:.0f}")
print(f"Avg Time: {baseline_results['avg_time_seconds']:.1f}s")


In [ ]:
# Now evaluate Deep Research approach
print("Running evaluation of Deep Research approach on AIME...")
print("Note: This will take longer as it performs iterative research\n")

deep_research_results = evaluate_approach_on_benchmark(
    aime_problems[:3],  # Evaluate same 3 problems
    approach="deep_research",
    benchmark_name="AIME-2024-Demo"
)

print("\n" + "="*80)
print("Deep Research Results Summary:")
print("="*80)
print(f"Accuracy: {deep_research_results['accuracy']:.1f}%")
print(f"Correct: {deep_research_results['correct']}/{deep_research_results['total_problems']}")
print(f"Avg Tokens: {deep_research_results['avg_tokens']:.0f}")
print(f"Avg Time: {deep_research_results['avg_time_seconds']:.1f}s")


In [ ]:
# Compare the two approaches
comparison_data = {
    "Approach": ["Baseline", "Deep Research"],
    "Accuracy (%)": [baseline_results['accuracy'], deep_research_results['accuracy']],
    "Avg Tokens": [baseline_results['avg_tokens'], deep_research_results['avg_tokens']],
    "Avg Time (s)": [baseline_results['avg_time_seconds'], deep_research_results['avg_time_seconds']]
}

comparison_df_demo = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("Side-by-Side Comparison:")
print("="*80)
print(comparison_df_demo.to_string(index=False))

# Quick visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Accuracy
axes[0].bar(comparison_df_demo["Approach"], comparison_df_demo["Accuracy (%)"], color=['#3498db', '#e74c3c'])
axes[0].set_ylabel("Accuracy (%)")
axes[0].set_title("Accuracy Comparison")
axes[0].set_ylim(0, 100)

# Tokens
axes[1].bar(comparison_df_demo["Approach"], comparison_df_demo["Avg Tokens"], color=['#3498db', '#e74c3c'])
axes[1].set_ylabel("Average Tokens")
axes[1].set_title("Token Usage Comparison")

# Time
axes[2].bar(comparison_df_demo["Approach"], comparison_df_demo["Avg Time (s)"], color=['#3498db', '#e74c3c'])
axes[2].set_ylabel("Average Time (s)")
axes[2].set_title("Time Comparison")

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "demo_comparison.png"), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Visualization saved to {RESULTS_DIR}/demo_comparison.png")


## SOTA Performance Comparison

Let's compare Deep Research with state-of-the-art models across all benchmarks.


In [ ]:
# Load SOTA performance data
sota_df = get_sota_performance_data()

print("SOTA Model Performance (Published Benchmarks):")
print("="*80)
display(sota_df)

# Create a comprehensive comparison visualization
benchmarks = ["AIME-2024", "MATH-500", "IMO", "FrontierMath", "HARP"]

fig = go.Figure()

for benchmark in benchmarks:
    if benchmark in sota_df.columns:
        fig.add_trace(go.Bar(
            name=benchmark,
            x=sota_df["Model"],
            y=sota_df[benchmark],
            text=sota_df[benchmark].round(1),
            textposition='auto',
        ))

fig.update_layout(
    title="SOTA Models Performance Across Math Benchmarks",
    xaxis_title="Model",
    yaxis_title="Accuracy (%)",
    barmode="group",
    height=600,
    xaxis_tickangle=-45,
    font=dict(size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

# Save
fig.write_html(os.path.join(RESULTS_DIR, "sota_baseline.html"))
print(f"\n✓ Saved SOTA baseline visualization")


## Conclusion and Key Findings

### Summary

This evaluation provides comprehensive insights into the Deep Research (TTD-DR) technique's performance on mathematical reasoning tasks compared to SOTA models.

### Key Takeaways

1. **Iterative Refinement**: Deep Research uses Test-Time Diffusion with iterative denoising
2. **External Knowledge**: Integrates web search for enhanced reasoning
3. **Draft-Guided Search**: Identifies knowledge gaps and performs targeted retrieval
4. **Quality Assessment**: Automatically evaluates draft quality for termination

### Trade-offs

- **Pros**: More comprehensive reasoning, access to external knowledge, systematic gap analysis
- **Cons**: Higher token usage, longer inference time

### Recommendations

Based on your specific use case:
- **When to use Deep Research**: Complex problems requiring multi-step reasoning and external knowledge
- **When to use baseline**: Simple problems where quick responses are needed

### Next Steps

1. Run the full evaluation with `NUM_PROBLEMS_PER_BENCHMARK = 30`
2. Test with different base models (GPT-4, Claude, etc.)
3. Tune `MAX_DEEP_RESEARCH_ITERATIONS` and `MAX_SOURCES` parameters
4. Analyze error patterns for specific problem types

### Files Generated

All results are saved in `deep_research_benchmark_results/`:
- `summary_report.json` - Complete evaluation data
- `*_results.json` - Per-benchmark detailed results
- `*.html` - Interactive visualizations
- `*.png` - Static charts
